# Productor consumidor con cola

En este ejemplo existe un buffer en la que el productor acumula los valores.
El bffer se ha creado como una cola finita, para mostrar el acceso exclusivo a esta con las funciones put y get con acceso en exclusión mutua y tiempo de espera limitado.
Se muestra el manejo de la excepción cola  vacía (queue.Empty) y cola llena (queue.Full).
Se ha implementado un retardo grande en los consumidores  para mostrar como se llena la cola pero como posteriormente
se despiertan los consumidores se muestra como se va consumiendo de forma ordenada.

In [6]:
import random
import threading
import time
import queue




class Buffer:
    def __init__(self,buffer):
        self.buffer=buffer
        
        
    def consumir(self):
        try:
            valor=buffer.get(block=True,timeout=20) #operación atómica, 
            #el tim-out limita la espera
            #pasado el tiempo se produce una excepcion
            texto="El {hilo} ha obtenido el valor {valor}.\n"
            print (texto.format(hilo=threading.currentThread().getName(), valor=valor))
        except queue.Empty:
            texto="El {hilo} se ha quedado sin valores.\n"
            print (texto.format(hilo=threading.currentThread().getName()))
            return None
        return valor
            
    def producir(self, valor):
        try:
            buffer.put(valor,block=True,timeout=3)# operación atómica
            texto="El {hilo} ha producido el valor {valor}.\n"
            print (texto.format(hilo=threading.currentThread().getName(), valor=valor))
        except queue.Full:
            texto="No ha espacio para guardar más valores, el {hilo} queda a la espera.\n"
            print (texto.format(hilo=threading.currentThread().getName()))
            buffer.put(valor,block=True)
            texto="El {hilo} ha producido el valor {valor}.\n"
            print (texto.format(hilo=threading.currentThread().getName(), valor=valor))
            
class ConsumidorBuffer(threading.Thread):
    def __init__(self, nombre, num_consultas, recurso):
        threading.Thread.__init__(self)
        self.name=nombre
        self.valor=None
        self.num_consultas=num_consultas
        self.recurso=recurso
    
    def run(self):
        print("Comienza "+self.name)
        for i in range(self.num_consultas):
            self.valor=self.recurso.consumir()
            time.sleep(20)
            
class ProductorBuffer(threading.Thread):
    def __init__(self, nombre, num_consultas, recurso):
        threading.Thread.__init__(self)
        self.name=nombre
        self.valor=None
        self.num_consultas=num_consultas
        self.recurso=recurso
    
    def run(self):
        print("Comienza ", self.name)
        for i in range(self.num_consultas):
            valor=random.random()
            self.recurso.producir(valor)
            time.sleep(1)
            
            
if __name__=="__main__":
    
    buffer=queue.Queue(maxsize=2)
    
    monitor=Buffer (buffer)   
    consumidor1 = ConsumidorBuffer('Consumidor 1', 3, monitor)
    consumidor2 = ConsumidorBuffer('Consumidor 2', 3, monitor)
    productor = ProductorBuffer('Productor ', 5, monitor)
    
    consumidor1.start()
    time.sleep(3)
    consumidor2.start()
    time.sleep(2)

    productor.start()
    
    

Comienza Consumidor 1
Comienza Consumidor 2
Comienza  Productor 
El Productor  ha producido el valor 0.8109083926057918.

El Consumidor 1 ha obtenido el valor 0.8109083926057918.

El Productor  ha producido el valor 0.3216571148329661.

El Consumidor 2 ha obtenido el valor 0.3216571148329661.

El Productor  ha producido el valor 0.6605841817008868.

El Productor  ha producido el valor 0.7574924669810206.

No ha espacio para guardar más valores, el Productor  queda a la espera.

El Consumidor 1 ha obtenido el valor 0.6605841817008868.

El Productor  ha producido el valor 0.7677350650196143.

El Consumidor 2 ha obtenido el valor 0.7574924669810206.

El Consumidor 1 ha obtenido el valor 0.7677350650196143.

El Consumidor 2 se ha quedado sin valores.

